In [4]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString, Comment
import html5lib

In [5]:
_bongdaplus = {
        "home_page":"https://bongdaplus.vn/",
        "urls":{
            "sport":
            {
             "url":"https://kenh14.vn/sport.chn",
             "sub-category":{
                0:{
                  "name":"Hậu trường bóng đá",
                  "url":"https://bongdaplus.vn/hau-truong-bong-da",
                  "cate_id":38,
                  "url_list" : []}
             }
            }
        }
    }


In [6]:
_bongdaplus

{'home_page': 'https://bongdaplus.vn/',
 'urls': {'sport': {'url': 'https://kenh14.vn/sport.chn',
   'sub-category': {0: {'name': 'Hậu trường bóng đá',
     'url': 'https://bongdaplus.vn/hau-truong-bong-da',
     'cate_id': 38,
     'url_list': []}}}}}

In [4]:
def get_list_url(cate_url):
    response = requests.get(cate_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    list_url = soup.find('div', attrs = {'data-marked-zoneid':'kenh14_detail_d3'}).find_all('li')
    list = []
    for i in list_url:
        try:
            path = i.find('a')['href']
            url = 'https://kenh14.vn'+ path
            list.append(url)
        except TypeError:
            continue
    return list
        

In [5]:
get_list_url('https://kenh14.vn/sport/bong-da.chn')


['https://kenh14.vn/chuyen-tinh-tai-tieng-cua-sao-bong-da-nguoi-trang-hoa-ke-giat-bo-dong-nghiep-20231125084606905.chn',
 'https://kenh14.vn/lap-sieu-pham-tu-khoang-cach-hon-30m-ronaldo-di-vao-lich-su-20231125084207354.chn',
 'https://kenh14.vn/khoanh-khac-tuan-hai-hoa-thanh-david-beckham-viet-nam-ghi-ban-tu-giua-san-20231125005616298.chn',
 'https://kenh14.vn/clb-tphcm-chinh-thuc-len-tieng-ve-lum-xum-no-luong-cau-thu-20231125005311325.chn',
 'https://kenh14.vn/thua-doi-thu-hang-15-the-gioi-nguyen-thuy-linh-dung-buoc-o-giai-china-masters-20231124174301543.chn']

In [25]:
#convert time string to the right format
def convert_string(input_str):
    # Extract the date part using regular expression
    pattern = r'\d{2}/\d{2}/\d{4}'
    date_part = re.search(pattern, input_str).group()
    match = re.search(pattern, input_str)
    if match:
        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_part, '%d/%m/%Y')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    else:
        return ""
# return the expected content in html format
def get_content_kenh14(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    article =  soup.find('div', class_ = "klw-new-content")
    children = article.find_all(recursive=False)
    related_news =  article.find('div', class_ = 'knc-relate-wrapper relationnews')
    related_news.decompose()
    for i in children[-6:]:
        i.decompose() 
    caption_text_list =article.find_all('div', class_="PhotoCMS_Caption")
    img_list = article.find_all(['img','video'])
    n_img = len(img_list)
    for i in range(0,n_img):
        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        caption_text = NavigableString(caption_text_list[i].text)
        caption_end = NavigableString("[/caption]")
        caption_text_list[i].decompose()
        # Insert the custom tags and caption text around the <img> tag
        img_list[i].insert_before(caption_start)
        img_list[i].insert_after(caption_end)
        try:
            img_list[i].insert_after(caption_text) 
        except IndexError as e:
            print(e)        #print(len(caption_text_list))
    for script_or_style in article(['script', 'style']):
            script_or_style.decompose() 
    for i in article.find_all(recursive = True):
        try:
            del i['onclick']
            del i['id']
            del i['class']
            del i['style']
        except AttributeError:
            continue
        except TypeError:
            continue            
    tags_to_remove = article.find_all(['a', 'span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
            # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip() 
    for i in article.find_all('img'):
        i['class'] = "aligncenter"
        i['width'] = 800
        i['height'] = 400
    for video in article.find_all('video'):
        video.parent.decompose()
        #video.decompose(
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: kenh14.vn"  # Set the content of <i> tag
    # Append the <i> tag as the last child of the <article> tag
    article.append(source_tag)
    for i in article.find_all('div', {'align': 'center'}):
        i.decompose()
    for element in article.find_all(recursive = True,string=True):
        if isinstance(element, NavigableString) and element.strip() == '':
            element.extract()
    #remove element withou child element or have children element which is empty
    for i in article.find_all(recursive = True):
        if i.children == None and i.string == None:
            i.decompose()
    for i in article.find_all(recursive = True):
        try:
            i.text.trip()
        except AttributeError as e:
            continue
    return article
    
def get_post(url):
    try:
        response = requests.get(url)
        time.sleep(5)
        soup = BeautifulSoup(response.content, 'html5lib')
        content = get_content_kenh14(url)
        post_time = soup.find('span', class_ = 'kbwcm-time').text.strip()
        published_date = convert_string(post_time)
        title = soup.find('h1').text.strip()
        return content,title,published_date
    except AttributeError as e:
        print(e)
#convert time string to datetime object
def convert_time_string(posted_date):
    pattern = r'\d{2}/\d{2}/\d{4}'
    match = re.search(pattern, posted_date)

    if match:
        date_string = match.group()

        # Convert to datetime object
        datetime_obj = datetime.strptime(date_string, "%d/%m/%Y")
        return datetime_obj
    else:
        return ''
def get_list_url(cate_url):
    response = requests.get(cate_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    list_url = soup.find('div', attrs = {'data-marked-zoneid':'kenh14_detail_d3'}).find_all('li')
    list = []
    for i in list_url:
        try:
            path = i.find('a')['href']
            url = 'https://kenh14.vn'+ path
            list.append(url)
        except TypeError:
            continue
    return list
def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time())
    for i in urls:
        response = requests.get(i)
        soup = BeautifulSoup(response.content, 'html5lib')
        try:
            date_posted = soup.find('span', class_ = 'kbwcm-time').text.strip()
            date_posted_norm = convert_time_string(date_posted)
            if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
                filtered_urls.append(i)
                print(i)
        except AttributeError as e:
            print(e)
            break
    return filtered_urls
def add_list(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):  
            urls = get_list_url(web_json_obj['urls'][i]['sub-category'][j]['url'])
            print(i,j,web_json_obj['urls'][i]['sub-category'][j]['url'])
            web_json_obj['urls'][i]['sub-category'][j]['url_list'] = filter_list(urls)

def add_post(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):
            web_json_obj['urls'][i]['sub-category'][j]['content'] = {}
            list_key = [v for v in range(0,len(web_json_obj['urls'][i]['sub-category'][j]['url_list']))]
            for u in list_key:
                web_json_obj['urls'][i]['sub-category'][j]['content'][u] = {}
                if u != "":
                    web_json_obj['urls'][i]['sub-category'][j]['content'][u]['text'] ,web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['published_date'] = get_post(web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
                    print(i,j,web_json_obj['urls'][i]['sub-category'][j]['cate_id'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
                
def get_news_kenh14():
    _kenh14 = {
            "home_page":"https://kenh14.vn/",
            "urls":{
                "sport":
                {
                 "url":"https://kenh14.vn/sport.chn",
                 "sub-category":{  
                    0:{"name":"Hậu trường",
                     "url":"https://kenh14.vn/sport/hau-truong.chn",
                     "cate_id":38,
                      "url_list" : []},
                    1:{"name":"Esports",
                     "url":"https://kenh14.vn/sport/esports.chn",
                     "cate_id":61,
                      "url_list" : []}
                 }
                }
            }
        }
#
    add_list(_kenh14)
    add_post(_kenh14)
    return _kenh14
def send_post_to_5goals(title,content,category_id,published_date):
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "category_id": category_id,
        "token": '5goalvodichcmnl',  # Replace with your actual access token
        "published_date": published_date,
        "domain":"kenh14"
          # Replace with the actual category ID as required
    }
    
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")
def main():
    _kenh14 = get_news_kenh14()
    for i in list(_kenh14['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(_kenh14['urls'][i]['sub-category']):
            url_list =  _kenh14['urls'][i]['sub-category'][j]['url_list']
            for t in range(0,len(url_list)):
                content = _kenh14['urls'][i]['sub-category'][j]['content'][t]['text']
                title = _kenh14['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = _kenh14['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = _kenh14['urls'][i]['sub-category'][j]['cate_id']
                send_post_to_5goals(title,str(content), cate_id, published_date)
                time.sleep(5)
    return _kenh14


In [3]:
send_post_to_5goals('phim mới', '<p>test </p>', 'test title', '2023-11-22')

The post was successfully created.
Response: {"success":false,"error_message":"B\u00e0i vi\u1ebft \u0111\u00e3 t\u1ed3n t\u1ea1i."}


In [188]:
_kenh14 = main()

sport 0 https://kenh14.vn/sport/hau-truong.chn
sport 1 https://kenh14.vn/sport/esports.chn
https://kenh14.vn/noi-tieng-chieu-fan-faker-phai-chang-se-co-the-so-huu-t1-ahri-day-nong-bong-20231125131654448.chn
sport 1 61 Esports Esports Nổi tiếng “chiều fan”, Faker phải chăng sẽ có thể sở hữu T1 Ahri đầy "nóng bỏng"? https://kenh14.vn/noi-tieng-chieu-fan-faker-phai-chang-se-co-the-so-huu-t1-ahri-day-nong-bong-20231125131654448.chn
The post was successfully created.
Response: {"success":true,"post_id":63362}


In [182]:
_kenh14

{'home_page': 'https://kenh14.vn/',
 'urls': {'sport': {'url': 'https://kenh14.vn/sport.chn',
   'sub-category': {0: {'name': 'Hậu trường',
     'url': 'https://kenh14.vn/sport/hau-truong.chn',
     'cate_id': 38,
     'url_list': [],
     'content': {}},
    1: {'name': 'Esports',
     'url': 'https://kenh14.vn/sport/esports.chn',
     'cate_id': 61,
     'url_list': ['https://kenh14.vn/noi-tieng-chieu-fan-faker-phai-chang-se-co-the-so-huu-t1-ahri-day-nong-bong-20231125131654448.chn'],
     'content': {0: {'text': <div class="klw-new-content"><div><div><div><div></div><div></div></div></div></div><h2>
                                           Cộng đồng LMHT mới đây đã nghĩ ra một ý tưởng đầy táo bạo đối với trang phục vô địch thế giới của Faker.
                                       </h2><div></div><div></div><div data-check-position="body_start"></div><div><!-- Kham pha --><p>Với việc cùng T1 giành danh hiệu CKTG 2023, Faker đã sở hữu tới 4 chức vô địch thế giới trong sự nghiệp của

In [186]:
for i in list(_kenh14['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
    for j in list(_kenh14['urls'][i]['sub-category']):
        url_list =  _kenh14['urls'][i]['sub-category'][j]['url_list']
        for t in range(0,len(url_list)):
            content = _kenh14['urls'][i]['sub-category'][j]['content'][t]['text']
            title = _kenh14['urls'][i]['sub-category'][j]['content'][t]['title']
            published_date = _kenh14['urls'][i]['sub-category'][j]['content'][t]['published_date']
            send_post_to_5goals(title,str(content), title, published_date)

The post was successfully created.
Response: {"success":true,"post_id":63360}


In [26]:
test = get_content_kenh14('https://kenh14.vn/bat-tron-khoanh-khac-doan-van-hau-va-doan-hai-my-di-bar-quay-het-minh-sau-khi-xa-vai-co-dau-chu-re-20231127072708131.chn')

In [27]:
print(test.prettify())

<div class="klw-new-content">
 <div>
  <div>
   <div>
    <div>
    </div>
    <div>
    </div>
   </div>
  </div>
 </div>
 <h2>
  Team qua đường bắt trọn hình ảnh của Đoàn Văn Hậu - Doãn Hải My cùng bạn bè đi "quẩy" sau khi tàn tiệc đám cưới.
 </h2>
 <div>
 </div>
 <div>
 </div>
 <div data-check-position="body_start">
 </div>
 <div>
  <!-- Kham pha -->
  <p>
   Tối ngày 26/11, đám cưới của hậu vệ Đoàn Văn Hậu và Doãn Hải My chiếm trọng mọi thông tin trên các nền tảng mạng xã hội. Sau 3 năm hẹn hò, cặp đôi chàng cầu thủ và nàng top 10 hoa hậu Việt Nam đã có một cái kết trọn vẹn. Sau khi tàn tiệc cưới, "xả vai" cô dâu chú rể, team qua đường đã bắt trọn hình ảnh của cặp đôi này xuống từ xe bạc tỉ vào một quán bar "quẩy" sau tiệc cưới cùng hội bạn bè. Văn Hậu và Hải My đến muộn nhất, đi cùng cặp đôi còn có bạn thân của cô dâu là diễn viên Bảo Hân. Trước đó bạn bè thân thiết của đôi trai tài gái sắc đã di chuyển ra bar trước ngay sau khi tiệc cưới kết thúc.
  </p>
  <div type="Photo">
  </

In [13]:
response = requests.get('https://kenh14.vn/noi-tieng-chieu-fan-faker-phai-chang-se-co-the-so-huu-t1-ahri-day-nong-bong-20231125131654448.chn')
soup = BeautifulSoup(response.content, 'html.parser')

In [14]:
article =  soup.find('div', class_ = "klw-body-top")

In [150]:
soup.find('span', class_ = 'kbwcm-time')

<span class="kbwcm-time" title="2023-11-24T09:00:00">09:00 24/11/2023</span>

In [15]:
len( article.find_all(['img','video'])[:-1])

4

In [16]:
article.find_all('div', class_ = 'PhotoCMS_Caption')

[<div class="PhotoCMS_Caption"><p data-placeholder="[nhập chú thích]">Faker mới đây đã chính thức giành được chức vô địch thế giới thứ 4 trong sự nghiệp</p></div>,
 <div class="PhotoCMS_Caption"><p data-placeholder="[nhập chú thích]">Faker từng chia sẻ rằng: “Tôi sẽ chọn vị tướng được nhiều người hâm mộ mong muốn”</p></div>,
 <div class="PhotoCMS_Caption"><p data-placeholder="[nhập chú thích]">Ý kiến táo bạo về trang phục Ahri Tiệc Bể Bơi với hình xăm logo T1 được nhiều game thủ đồng tình</p></div>,
 <div class="PhotoCMS_Caption"><p data-placeholder="[nhập chú thích]">Nhân viên Riot từng chia sẻ rằng họ chỉ làm trang phục Ahri Tiệc Bể Bơi nếu quá thiếu tiền</p></div>]

In [129]:
response = requests.get('https://kenh14.vn/doan-hai-my-nhan-doa-hoa-dac-biet-tu-doan-van-hau-truoc-tiec-cuoi-tai-ha-noi-20231124084630322.chn')
soup = BeautifulSoup(response.content, 'html.parser')
article =  soup.find('div', class_ = "klw-new-content")
children = article.find_all(recursive=False)
related_news =  article.find('div', class_ = 'knc-relate-wrapper relationnews')
related_news.decompose()
for i in children[-6:]:
    i.decompose() 
caption_text_list =article.find_all( class_="PhotoCMS_Caption")
img_list = article.find_all(['img','video'])[:-1]
n_img = len(img_list)
for i in range(0,n_img):
    caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
    caption_text = NavigableString(caption_text_list[i].text)
    caption_end = NavigableString("[/caption]")
    caption_text_list[i].decompose()
        # Insert the custom tags and caption text around the <img> tag
    img_list[i].insert_before(caption_start)
    img_list[i].insert_after(caption_end)
    try:
        img_list[i].insert_after(caption_text) 
    except IndexError as e:
        print(e)        #print(len(caption_text_list))
for script_or_style in article(['script', 'style']):
        script_or_style.decompose() 
for i in article.find_all(recursive = True):
    try:
        del i['onclick']
        del i['id']
        del i['class']
        del i['style']
    except AttributeError:
        continue
    except TypeError:
        continue            
tags_to_remove = article.find_all(['a', 'span'])
for tag in tags_to_remove:
    # Extract the text from the tag
    tag_text = tag.get_text()
        # Replace the tag with its text content
    tag.replace_with(tag_text)
    tag.text.strip() 
for i in article.find_all('img'):
    i['class'] = "aligncenter"
    i['width'] = 800
    i['height'] = 400
source_tag = soup.new_tag('i') 
source_tag.string = "Nguồn: kenh14.vn"  # Set the content of <i> tag
# Append the <i> tag as the last child of the <article> tag
article.append(source_tag)
for i in article.find_all('div', {'align': 'center'}):
    i.decompose()
for element in article.find_all(recursive = True,string=True):
    if isinstance(element, NavigableString) and element.strip() == '':
        element.extract()
#remove element withou child element or have children element which is empty
for i in article.find_all(recursive = True):
    if i.children == None and i.string == None:
        i.decompose()
for i in article.find_all(recursive = True):
    try:
        i.text.trip()
    except AttributeError as e:
        continue

In [130]:
for i in article.find_all(recursive = True):
    if i.string is None and i.text == "" and i.name != 'img' :
        print(i)
        

<div><div><div><div></div><div></div></div></div></div>
<div><div><div></div><div></div></div></div>
<div><div></div><div></div></div>
<div></div>
<div></div>
<div data-check-position="body_start"></div>
<div><img alt="Doãn Hải My nhận đóa hoa đặc biệt từ Đoàn Văn Hậu trước tiệc cưới tại Hà Nội - Ảnh 17." class="aligncenter" data-original="https://kenh14cdn.com/203336854389633024/2023/11/24/photo-1-1700790213567900924736.jpeg" h="1334" height="400" photoid="652324223701291008" rel="lightbox" src="https://kenh14cdn.com/thumb_w/620/203336854389633024/2023/11/24/photo-1-1700790213567900924736.jpeg" title="Doãn Hải My nhận đóa hoa đặc biệt từ Đoàn Văn Hậu trước tiệc cưới tại Hà Nội - Ảnh 17." type="photo" w="2000" width="800"/></div>
<zone></zone>
<zone></zone>


In [42]:
print(article.find_all(recursive = True)[2].string)

None


In [160]:
caption_text_list =article.find_all('div', class_ ="PhotoCMS_Caption")
caption_text_list[0].text

''

In [153]:
len(caption_text_list)

16

In [146]:
len(article.find_all(['img','video']))

18

In [120]:
article

<div class="klw-new-content"><div><div><div><div></div><div></div></div></div></div><h2>
                                    Cặp đôi hot nhất nhì tháng 11 đồng loạt "xả" bộ ảnh dành riêng cho tiệc cưới ở Hà Nội.
                                </h2><div></div><div></div><div data-check-position="body_start"></div><div><!-- Kham pha --><div type="Photo"><div>[caption id="" align="aligncenter" width="800"]<img alt="Doãn Hải My nhận đóa hoa đặc biệt từ Đoàn Văn Hậu trước tiệc cưới tại Hà Nội - Ảnh 1." class="aligncenter" data-original="https://kenh14cdn.com/203336854389633024/2023/11/24/photo-17-17007902400121501122882.jpeg" h="1049" height="400" photoid="652324335252873216" rel="lightbox" src="https://kenh14cdn.com/thumb_w/620/203336854389633024/2023/11/24/photo-17-17007902400121501122882.jpeg" title="Doãn Hải My nhận đóa hoa đặc biệt từ Đoàn Văn Hậu trước tiệc cưới tại Hà Nội - Ảnh 1." type="photo" w="700" width="800"/>Ngày 26/11, hậu vệ Đoàn Văn Hậu và bà xã Doãn Hải My sẽ tổ chức tiệc

In [123]:

# Example HTML content with nested elements
html_content = """
<div>
    <p>First paragraph.</p>
    <div>
        <span>Nested span</span>
        <p>Nested paragraph</p>
    </div>
</div>
"""
soup = BeautifulSoup(html_content, 'html.parser')
def print_all_children(element, depth=0):
    children = []

    """Recursively print all children of an element."""
    for child in element.children:
        if hasattr(child, 'name') and child.name is None:
            print("--" * depth + str(child.name))
            children.append(str(child.name))
            #print_all_children(child, depth + 1)
    return children

# Find a specific element, e.g., the first <div> tag

# Print all children of the <div> tag
print_all_children(article)

None


['None']

In [124]:
children = print_all_children(article)

None


In [125]:
children

['None']